# High density genotypes of French Sheep populations
describing `High density genotypes of French Sheep populations.zip`

In [1]:
import csv
import itertools

from collections import Counter

import pandas as pd
from plinkio import plinkfile
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from tqdm.notebook import tqdm

from src.features.utils import countries
from src.features.smarterdb import VariantSheep, global_connection, Dataset

get dataset informations relying on project classes

In [2]:
global_connection()
dataset = Dataset.objects.filter(file="High density genotypes of French Sheep populations.zip").get()
tqdm.pandas()

Display directory content (of selected dataset):

In [3]:
for item in dataset.working_dir.iterdir():
    print(item)

/home/paolo/Projects/SMARTER-database/data/interim/604f75a61a08c53cebd09b58/Populations_infos.xlsx
/home/paolo/Projects/SMARTER-database/data/interim/604f75a61a08c53cebd09b58/info.txt
/home/paolo/Projects/SMARTER-database/data/interim/604f75a61a08c53cebd09b58/OAR3
/home/paolo/Projects/SMARTER-database/data/interim/604f75a61a08c53cebd09b58/frenchsheep_HD.bim
/home/paolo/Projects/SMARTER-database/data/interim/604f75a61a08c53cebd09b58/Populations_infos_fix.xlsx
/home/paolo/Projects/SMARTER-database/data/interim/604f75a61a08c53cebd09b58/frenchsheep_HD.bed
/home/paolo/Projects/SMARTER-database/data/interim/604f75a61a08c53cebd09b58/OAR4
/home/paolo/Projects/SMARTER-database/data/interim/604f75a61a08c53cebd09b58/frenchsheep_HD.fam


In [4]:
with open(dataset.working_dir / "info.txt") as handle:
    print(handle.read())

https://zenodo.org/record/237116#.XlUezRdG3OQ


This dataset is in plink binary format, with population info in a xlsx file. Data were downloaded from a zenodo project

In [5]:
with open(dataset.working_dir / "Populations_infos.xlsx", "rb") as handle:
    infos = pd.read_excel(handle)
infos.head()

,Code,Population Name,Link,Latitude,Longitude,Color,POP_GROUP_CODE,POP_GROUP_NAME
0,BER,Berrichon du Cher,http://en.france-genetique-elevage.org/Berrich...,47.081012,2.398782,NaN,NORTH,NORTH
1,BMC,Blanc du Massif Central,http://en.france-genetique-elevage.org/Blanche...,44.517611,3.501873,NaN,SOUTH,SOUTH
2,CDL,Causses du Lot,http://en.france-genetique-elevage.org/Causse-...,44.799383,1.617901,NaN,SOUTH,SOUTH
3,CHA,Mouton Charollais,http://en.france-genetique-elevage.org/Charoll...,46.435442,4.277004,NaN,NORTH,NORTH
4,CHR,Charmoise,http://en.france-genetique-elevage.org/Charmoi...,47.390249,1.254324,NaN,NORTH,NORTH


Data have informations on breed (with code) and GPS coordinates. What about SNPs coordinates? open the `.bim` file (which is in txt format) and test SNPs

In [6]:
counter = Counter()
with open(dataset.working_dir / "frenchsheep_HD.bim") as f:
    reader = csv.reader(f, delimiter="\t")
    print("Reading first 200 lines of bim file")
    for line in itertools.islice(reader, 200):
        qs = VariantSheep.objects(name=line[1])
        if qs.count() > 0:
            variant = qs.get()
            counter.update([variant.name])
            location = next(filter(lambda loc: loc.imported_from == "SNPchiMp v.3", variant.locations))
            if line[0] != location.chrom or int(line[3]) != location.position:
                print(f"snp {line[1]} with different positions: {line[0]}:{line[3]}<>{location.chrom}:{location.position}")


Reading first 200 lines of bim file


Coordinates seems to match the latest *SNPchiMp v.3* database

## Getting info on breeds

Open genotypes with `plinkio` and get information on samples:

In [7]:
plink_file = plinkfile.open( str(dataset.working_dir / "frenchsheep_HD") )
sample_list = plink_file.get_samples( )

Get `fid` from samples

In [8]:
fids = set([sample.fid for sample in sample_list])
fids = pd.Series(list(fids), name="Code")

In [9]:
merged = pd.merge(fids,infos, on="Code")
merged.loc[:, ["Code", "Population Name"]]

,Code,Population Name
0,IDF,Île de France
1,ROM,Romanov
2,CDL,Causses du Lot
3,SUF,Suffolk
4,LAC,Lacaune (milk)
5,MER,Mérinos d'Arles
6,PAS,Préalpes du Sud
7,MTR,Manech Tête Rouge
8,LAM,Lacaune (meat)
9,OUE,Ouessant


## Check dataset countries
Test for sample countries of origin

In [10]:
# add a column for the query
infos["coordinates"] = infos["Latitude"].map(str) + ", " + infos["Longitude"].map(str)

# define the reverse geocoding
locator = Nominatim(user_agent="myGeocoder", timeout=10)
rgeocode = RateLimiter(locator.reverse, min_delay_seconds=0.1)
def get_country(coordinate):
    data = rgeocode(coordinate, language="English")
    if data:
        country_code = data.raw['address']['country_code']
        return countries.get(alpha_2=country_code).name
    else:
        return data

# find countries
infos["Country"] = infos["coordinates"].progress_apply(get_country)

# drop the coordinates column
infos.drop('coordinates', axis=1, inplace=True)

  0%|          | 0/27 [00:00<?, ?it/s]

manipulate dataframe and save the final dataset:

In [11]:

infos["Species"] = "Ovis aries"
infos.loc[infos['Population Name'] == "European Mouflon", "Species"] = "Ovis aries musimon"
infos.loc[infos['Population Name'] == "European Mouflon", "Population Name"] = "European mouflon"

Fix also the `Mérinos d'Arles` code:

In [12]:
infos.loc[infos['Population Name'] == "Mérinos d'Arles", "Code"] = "ARL"
infos.to_excel(dataset.working_dir / "Populations_infos_fix.xlsx", index=False)